# EDA on Light Training, Time of Year(Covid Shutdown), Water Temperature

In [186]:
#Light Training began on 2/15/18
#3/14/20 - 5/9/20 Closed to Public

In [187]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
import scipy.stats as st

In [188]:
eat = pd.read_excel("SharkData.xlsx", sheet_name = 0)
targets = pd.read_excel("SharkData.xlsx", sheet_name = 1)
drops = pd.read_excel("SharkData.xlsx", sheet_name = 2)
other_factors = pd.read_excel("SharkData.xlsx", sheet_name = 3)

Cleaning for each each Dataframe

In [189]:
eat = eat.iloc[1:,:-1]
targets = targets.iloc[1:,:-1]
drops = drops.iloc[1:,:-1]
eat.rename(columns = {"Unnamed: 14":'Etc. Comments', "Pieces Eaten": "Date"}, inplace = True)
targets.rename(columns = {"Targets": "Date"}, inplace = True)
drops.rename(columns = {"Drops": "Date"}, inplace = True)

#get rid of null values at end of dataset
eat = eat.head(582)
targets = targets.head(582)
drops = drops.head(582)

# Change date to just date format, not datetime
eat['Date'] = pd.to_datetime(eat['Date']).dt.date
targets['Date'] = pd.to_datetime(targets['Date']).dt.date
drops['Date'] = pd.to_datetime(drops['Date']).dt.date


eat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,Etc. Comments
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN


In [190]:
targets.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
1,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,3.0,3.0,0.0,22.0
2,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,2.0,2.0,0.0,30.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0,0.0,3.0,8.0
4,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,15.0
5,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,5.0,1.0,3.0,26.0


In [191]:
drops.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
1,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0
2,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,10.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0
4,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
5,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0


Add the temperature to each Dataframe

In [192]:
other_factors['Temperature'].count()

582

In [193]:
#eat['Temperature'] = other_factors['Temperature'].values
#targets['Temperature'] = other_factors['Temperature'].values
#drops['Temperature'] = other_factors['Temperature'].values

In [194]:
eat

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,Etc. Comments
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,2020-09-22,6.0,0.0,1.0,2.0,6.0,0.0,4.0,1.0,0.0,0.0,5.0,1.0,26.0,NaN
579,2020-09-24,0.0,9.0,0.0,7.0,5.0,5.0,7.0,0.0,1.0,2.0,2.0,6.0,44.0,NaN
580,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,2.0,3.0,3.0,12.0,NaN
581,2020-09-27,5.0,6.0,3.0,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN


Divide data according to Light Training vs Non-Light Training
-Light training starts on Row 42

In [195]:
eat_no_training = eat.head(41)
eat_training = eat[42:582]
targets_no_training = targets.head(41)
targets_training = targets[42:582]
drops_no_training = drops.head(41)
drops_training = drops[42:582]

In [196]:
drops_training

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
43,2018-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
44,2018-02-18,1.0,2.0,1.0,0.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,7.0
45,2018-02-20,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,7.0
46,2018-02-22,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0
47,2018-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,2020-09-22,0.0,0.0,1.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0
579,2020-09-24,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,7.0
580,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
581,2020-09-27,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,3.0


I am going to seperate the data into three different groups based on Pre-covid(1-470), covid(471-499), post_covid(500-582)

In [197]:
eat_pre = eat[0:470]
eat_covid = eat[471:499]
eat_post = eat[500:582]
targets_pre = targets[1:470]
targets_covid = targets[471:499]
targets_post = targets[500:582]
drops_pre = drops[1:470]
drops_covid = drops[471:499]
drops_post = drops[500:582]

# Eat EDA

In [198]:
all_sharks_pre = eat_pre[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_covid = eat_covid[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_post = eat_post[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]

In [199]:
all_mean_pre = round(all_sharks_pre.mean(), 3)
all_mean_covid = round(all_sharks_covid.mean(), 3)
all_mean_post = round(all_sharks_post.mean(), 3)
all_mean_pre

Ross        3.456
Chandler    2.811
BT1         2.088
BT2         2.691
BT3         2.323
BT4         2.703
BT5         3.040
GR1         1.266
GR2         1.391
GR3         1.909
GR4         1.635
GR5         1.683
dtype: float64

In [200]:
all_mean_covid

Ross        2.609
Chandler    2.652
BT1         1.826
BT2         4.304
BT3         1.435
BT4         2.696
BT5         2.565
GR1         1.048
GR2         1.000
GR3         1.762
GR4         1.286
GR5         1.143
dtype: float64

In [201]:
all_mean_post

Ross        4.452
Chandler    3.371
BT1         1.871
BT2         3.274
BT3         2.839
BT4         3.323
BT5         3.468
GR1         1.869
GR2         1.279
GR3         1.574
GR4         1.525
GR5         2.230
dtype: float64

In [202]:
# Graph Mean Number of Fish Eaten For Each Shark for each time pre, during, post covid.

eaten_means = {'Shark': all_sharks_pre.columns,
               'Pre_Mean': all_mean_pre,
               'Covid_Mean': all_mean_covid,
               'Post_Mean': all_mean_post}

eaten_means_df = pd.DataFrame(eaten_means, columns = ['Shark', 'Pre_Mean', 'Covid_Mean','Post_Mean'])

In [203]:
pre_eaten_means_sorted = eaten_means_df.sort_values(by=['Pre_Mean'], ascending = False)
covid_eaten_means_sorted = eaten_means_df.sort_values(by=['Covid_Mean'], ascending = False)
post_eaten_means_sorted = eaten_means_df.sort_values(by=['Post_Mean'], ascending = False)

In [204]:
fig = px.histogram(pre_eaten_means_sorted, x="Shark", y="Pre_Mean", 
                   title = "Mean Number of Fish Eaten by Shark before Covid",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [205]:

fig = px.histogram(covid_eaten_means_sorted, x="Shark", y="Covid_Mean", 
                   title = "Mean Number of Fish Eaten by Shark during Covid",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [206]:

fig = px.histogram(post_eaten_means_sorted, x="Shark", y="Post_Mean", 
                   title = "Mean Number of Fish Eaten by Shark after Covid",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [207]:
# Shark feeding vector pre covid
total_sharks_feeding_pre = eat_pre[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_pre = eat_pre[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_pre = eat_pre[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_pre = eat_pre[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_pre = eat_pre[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_pre = eat_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark feeding vector during covid
total_sharks_feeding_covid = eat_covid[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_covid = eat_covid[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_covid = eat_covid[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_covid = eat_covid[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_covid = eat_covid[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_covid = eat_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark feeding vector post covid
total_sharks_feeding_post = eat_post[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_post = eat_post[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_post = eat_post[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_post = eat_post[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_post = eat_post[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_post = eat_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_feeding_post

501     7
502    12
503    12
504     5
505     7
       ..
578    12
579    12
580     5
581     7
582    12
Length: 82, dtype: int64

In [208]:
eat_pre["male"] = eat_pre[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_pre["female"] = eat_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
eat_covid["male"] = eat_covid[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_covid["female"] = eat_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
eat_post["male"] = eat_post[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_post["female"] = eat_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

eat_pre["All_GR"] = eat_pre["GR1"] + eat_pre["GR2"] + eat_pre["GR3"]+ eat_pre["GR4"] + eat_pre["GR5"]
eat_pre["All_BT"] = eat_pre["BT1"] + eat_pre["BT2"] + eat_pre["BT3"]+ eat_pre["BT4"] + eat_pre["BT5"]
eat_pre["All_SS"] = eat_pre["Ross"] + eat_pre["Chandler"]

eat_covid["All_GR"] = eat_covid["GR1"] + eat_covid["GR2"] + eat_covid["GR3"]+ eat_covid["GR4"] + eat_covid["GR5"]
eat_covid["All_BT"] = eat_covid["BT1"] + eat_covid["BT2"] + eat_covid["BT3"]+ eat_covid["BT4"] + eat_covid["BT5"]
eat_covid["All_SS"] = eat_covid["Ross"] + eat_covid["Chandler"]

eat_post["All_GR"] = eat_post["GR1"] + eat_post["GR2"] + eat_post["GR3"]+ eat_post["GR4"] + eat_post["GR5"]
eat_post["All_BT"] = eat_post["BT1"] + eat_post["BT2"] + eat_post["BT3"]+ eat_post["BT4"] + eat_post["BT5"]
eat_post["All_SS"] = eat_post["Ross"] + eat_post["Chandler"]

<ipython-input-208-6d17c63af8cb>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-208-6d17c63af8cb>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-208-6d17c63af8cb>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-208-6d1

In [209]:
#Average Mean Graphs Gender - Pre

all_sharks_eat_pre = eat_pre[["female","male"]]
all_sharks_eat_pre["female"] = all_sharks_eat_pre["female"] / female_sharks_feeding_pre
all_sharks_eat_pre["male"] = all_sharks_eat_pre["male"] / male_sharks_feeding_pre
eat_pre_means = pd.DataFrame(round(all_sharks_eat_pre.mean(), 3)).reset_index()
eat_pre_means.columns = ["Gender","Mean"]
eat_pre_means = eat_pre_means.sort_values(by = "Mean", ascending = False)
print(eat_pre_means) 
px.bar(eat_pre_means, x = "Gender", y = "Mean", title = "How often is the average shark eating for each gender before Covid?")

   Gender   Mean
1    male  2.385
0  female  2.120


<ipython-input-209-ec069a954aef>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-209-ec069a954aef>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [210]:
#Average Mean Graphs Gender - During Covid

all_sharks_eat_covid = eat_covid[["female","male"]]
all_sharks_eat_covid["female"] = all_sharks_eat_covid["female"] / female_sharks_feeding_covid
all_sharks_eat_covid["male"] = all_sharks_eat_covid["male"] / male_sharks_feeding_covid
eat_covid_means = pd.DataFrame(round(all_sharks_eat_covid.mean(), 3)).reset_index()
eat_covid_means.columns = ["Gender","Mean"]
eat_covid_means = eat_covid_means.sort_values(by = "Mean", ascending = False)
print(eat_covid_means) 
px.bar(eat_covid_means, x = "Gender", y = "Mean", title = "How often is the average shark eating for each gender without patrons?")

   Gender   Mean
0  female  2.170
1    male  2.043


<ipython-input-210-7465f17d6367>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-210-7465f17d6367>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [211]:
#Average Mean Graphs Gender - Post

all_sharks_eat_post = eat_post[["female","male"]]
all_sharks_eat_post["female"] = all_sharks_eat_post["female"] / female_sharks_feeding_post
all_sharks_eat_post["male"] = all_sharks_eat_post["male"] / male_sharks_feeding_post
eat_post_means = pd.DataFrame(round(all_sharks_eat_post.mean(), 3)).reset_index()
eat_post_means.columns = ["Gender","Mean"]
eat_post_means = eat_post_means.sort_values(by = "Mean", ascending = False)
print(eat_post_means) 
px.bar(eat_post_means, x = "Gender", y = "Mean", title = "How often is the average shark eating for each gender after shutdwon")

<ipython-input-211-41e243802010>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-211-41e243802010>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



   Gender   Mean
1    male  2.770
0  female  2.259


In [212]:
eat_pre = eat_pre.drop('Etc. Comments', axis=1)
eat_pre

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,12.0,21.0,6.0,22.0,5.0
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,2.0,1.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,2.0,4.0,6.0,NaN,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,7.0,7.0,NaN,8.0,6.0
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,9.0,18.0,7.0,16.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2020-03-05,8.0,0.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,16.0,13.0,3.0,1.0,7.0,8.0
467,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,4.0,2.0,4.0,11.0,0.0,11.0,11.0,NaN,NaN
468,2020-03-08,0.0,8.0,9.0,0.0,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,21.0,19.0,2.0,NaN,13.0,8.0
469,2020-03-10,1.0,3.0,0.0,4.0,3.0,2.0,6.0,3.0,0.0,4.0,1.0,2.0,29.0,13.0,16.0,10.0,15.0,4.0


In [213]:
eat_pre.dtypes

Date         object
Ross        float64
Chandler    float64
BT1         float64
BT2         float64
BT3         float64
BT4         float64
BT5         float64
GR1         float64
GR2         float64
GR3         float64
GR4         float64
GR5         float64
Total:      float64
male        float64
female      float64
All_GR      float64
All_BT      float64
All_SS      float64
dtype: object

In [214]:
#Average Mean Graphs Per Species - Pre

all_sharks_eat_pre = eat_pre[["All_GR","All_BT","All_SS"]]
all_sharks_eat_pre["All_GR"] = all_sharks_eat_pre["All_GR"] / GR_sharks_feeding_pre
all_sharks_eat_pre["All_BT"] = all_sharks_eat_pre["All_BT"] / BT_sharks_feeding_pre
all_sharks_eat_pre["All_SS"] = all_sharks_eat_pre["All_SS"] / SS_sharks_feeding_pre
all_sharks_eat_pre.columns = ["Grey Reef","Black Tip","Sand Shark"]
eat_pre_means = pd.DataFrame(round(all_sharks_eat_pre.mean(), 3)).reset_index()
eat_pre_means.columns = ["Species","Mean"]
eat_pre_means = eat_pre_means.sort_values(by = "Mean", ascending = False)
print(eat_pre_means)
px.bar(eat_pre_means, x = "Species", y = "Mean", title = "Eat Means for Each species on average before covid")

      Species   Mean
2  Sand Shark  3.137
1   Black Tip  2.572
0   Grey Reef  1.577


<ipython-input-214-98dd1407732c>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-214-98dd1407732c>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-214-98dd1407732c>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [215]:
#Average Mean Graphs Per Species - During

all_sharks_eat_covid = eat_covid[["All_GR","All_BT","All_SS"]]
all_sharks_eat_covid["All_GR"] = all_sharks_eat_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_eat_covid["All_BT"] = all_sharks_eat_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_eat_covid["All_SS"] = all_sharks_eat_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_eat_covid.columns = ["Grey Reef","Black Tip","Sand Shark"]
eat_covid_means = pd.DataFrame(round(all_sharks_eat_covid.mean(), 3)).reset_index()
eat_covid_means.columns = ["Species","Mean"]
eat_covid_means = eat_covid_means.sort_values(by = "Mean", ascending = False)
print(eat_covid_means)
px.bar(eat_covid_means, x = "Species", y = "Mean", title = "Eat Means for Each species on average during shutdown")

      Species   Mean
2  Sand Shark  2.630
1   Black Tip  2.565
0   Grey Reef  1.248


<ipython-input-215-e4a025c86205>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-215-e4a025c86205>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-215-e4a025c86205>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [216]:
#Average Mean Graphs Per Sppost - Post

all_sharks_eat_post = eat_post[["All_GR","All_BT","All_SS"]]
all_sharks_eat_post["All_GR"] = all_sharks_eat_post["All_GR"] / GR_sharks_feeding_post
all_sharks_eat_post["All_BT"] = all_sharks_eat_post["All_BT"] / BT_sharks_feeding_post
all_sharks_eat_post["All_SS"] = all_sharks_eat_post["All_SS"] / SS_sharks_feeding_post
all_sharks_eat_post.columns = ["Grey Reef","Black Tip","Sand Shark"]
eat_post_means = pd.DataFrame(round(all_sharks_eat_post.mean(), 3)).reset_index()
eat_post_means.columns = ["Species","Mean"]
eat_post_means = eat_post_means.sort_values(by = "Mean", ascending = False)
print(eat_post_means)
px.bar(eat_post_means, x = "Species", y = "Mean", title = "Eat Means for Each species on average during shutdown")

<ipython-input-216-69c4528ee4e1>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-216-69c4528ee4e1>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-216-69c4528ee4e1>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



      Species   Mean
2  Sand Shark  3.911
1   Black Tip  2.955
0   Grey Reef  1.695


In [217]:
All_Sharks_eat_pre = eat_pre.iloc[:,1:]
All_Sharks_eat_pre["All_GR"] = All_Sharks_eat_pre["All_GR"] / GR_sharks_feeding_pre
All_Sharks_eat_pre["All_BT"] = All_Sharks_eat_pre["All_BT"] / BT_sharks_feeding_pre
All_Sharks_eat_pre["All_SS"] = All_Sharks_eat_pre["All_SS"] / SS_sharks_feeding_pre
All_Sharks_eat_pre["female"] = All_Sharks_eat_pre["female"] / female_sharks_feeding_pre
All_Sharks_eat_pre["male"] = All_Sharks_eat_pre["male"] / male_sharks_feeding_pre
All_Sharks_eat_pre["Total: "] = All_Sharks_eat_pre["Total: "] / total_sharks_feeding_pre
eat_pre_means = pd.DataFrame(round(All_Sharks_eat_pre.mean(), 3)).reset_index()
eat_pre_means.columns = ["Subset","Mean"]
eat_pre_means = eat_pre_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in eat_pre.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(eat_pre[column])-1, loc=np.mean(eat_pre[column]), scale=st.sem(eat_pre[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
eat_pre_means = pd.merge(eat_pre_means, ci, how = 'outer', on = "Subset")
#eat_pre_means = pd.merge(eat_pre_means,pd.melt(eat_pre.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = eat_pre_means[eat_pre_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in eat_pre_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eat_pre_means["Average"] = Average
max_y = (eat_pre_means["Mean"] + eat_pre_means["SE"]).max() 
#max_y = (eat_pre_means["Mean"] + eat_pre_means["Standard Deviation"]).max() 
px.bar(eat_pre_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Eat Means for each subset with Confidence Intervals").update_traces(textposition='outside')

In [218]:
# Mean number of fish eaten for Sandbar species Pre-Covid
Sandbar_pre = eat_pre['Ross'] + eat_pre['Chandler']
Sandbar_pre.mean() / 2

3.1370056497175143

In [219]:
# Mean of BT Pre-Covid
BT_pre = eat_pre['BT1'] + eat_pre['BT2'] + eat_pre['BT3'] + eat_pre['BT4'] + eat_pre['BT5']
BT_pre.mean() / 5

2.5715099715099714

In [220]:
# Mean of GR Pre-Covid
GR_pre = eat_pre['GR1'] + eat_pre['GR2'] + eat_pre['GR3'] + eat_pre['GR4'] + eat_pre['GR5']
GR_pre.mean() / 5

1.576770538243626

In [221]:
# Graph Mean Number of Fish Eaten For Each Species pre covid

Species_Eaten_Means_pre = {'Species': ['Blacktip', 'Gray Reef', 'Sandbar'],
        'Mean': [round(Sandbar_pre.mean()/2,3),round(BT_pre.mean()/5,3), round(GR_pre.mean()/5,3)]}

Species_Eaten_Means_pre_df = pd.DataFrame(Species_Eaten_Means_pre, columns = ['Species', 'Mean'])

Species_Eaten_Means_Sorted_pre = Species_Eaten_Means_pre_df.sort_values(by=['Mean'], ascending = False)

fig = px.histogram(Species_Eaten_Means_Sorted_pre, x="Species", y="Mean", 
                   title = "Mean Number of Fish Eaten Per Shark by Species before Covid",
                  labels={
                     "Species": "Species",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [222]:
# Mean number of fish eaten for Sandbar species During Covid
Sandbar_covid = eat_covid['Ross'] + eat_covid['Chandler']
Sandbar_covid.mean() / 2

2.630434782608696

In [223]:
# Mean of BT During Covid
BT_covid = eat_covid['BT1'] + eat_covid['BT2'] + eat_covid['BT3'] + eat_covid['BT4'] + eat_covid['BT5']
BT_covid.mean() / 5

2.5652173913043477

In [224]:
# Mean of GR during Covid
GR_covid = eat_covid['GR1'] + eat_covid['GR2'] + eat_covid['GR3'] + eat_covid['GR4'] + eat_covid['GR5']
GR_covid.mean() / 5

1.2476190476190476

In [225]:
# Graph Mean Number of Fish Eaten For Each Species during

Species_Eaten_Means_covid = {'Species': ['Blacktip', 'Gray Reef', 'Sandbar'],
        'Mean': [round(Sandbar_covid.mean()/2,3),round(BT_covid.mean()/5,3), round(GR_covid.mean()/5,3)]}

Species_Eaten_Means_covid_df = pd.DataFrame(Species_Eaten_Means_covid, columns = ['Species', 'Mean'])

Species_Eaten_Means_Sorted_covid = Species_Eaten_Means_covid_df.sort_values(by=['Mean'], ascending = False)

fig = px.histogram(Species_Eaten_Means_Sorted_covid, x="Species", y="Mean", 
                   title = "Mean Number of Fish Eaten Per Shark by Species during Covid",
                  labels={
                     "Species": "Species",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [226]:
# Mean number of fish eaten for Sandbar species Post-Covid
Sandbar_post = eat_post['Ross'] + eat_post['Chandler']
Sandbar_post.mean() / 2

3.911290322580645

In [227]:
# Mean of BT Post-Covid
BT_post = eat_post['BT1'] + eat_post['BT2'] + eat_post['BT3'] + eat_post['BT4'] + eat_post['BT5']
BT_post.mean() / 5

2.954838709677419

In [228]:
# Mean of GR Post-Covid
GR_post = eat_post['GR1'] + eat_post['GR2'] + eat_post['GR3'] + eat_post['GR4'] + eat_post['GR5']
GR_post.mean() / 5

1.6950819672131145

In [229]:
# Graph Mean Number of Fish Eaten For Each Species Post Covid

Species_Eaten_Means_post = {'Species': ['Blacktip', 'Gray Reef', 'Sandbar'],
        'Mean': [round(Sandbar_post.mean()/2,3),round(BT_post.mean()/5,3), round(GR_post.mean()/5,3)]}

Species_Eaten_Means_post_df = pd.DataFrame(Species_Eaten_Means_post, columns = ['Species', 'Mean'])

Species_Eaten_Means_Sorted_post = Species_Eaten_Means_post_df.sort_values(by=['Mean'], ascending = False)

fig = px.histogram(Species_Eaten_Means_Sorted_post, x="Species", y="Mean", 
                   title = "Mean Number of Fish Eaten Per Shark by Species Post-Covid",
                  labels={
                     "Species": "Species",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [230]:
#Organizing by gender pre, during, and post covid
eat_pre["male"] = eat_pre[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_pre["female"] = eat_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
eat_covid["male"] = eat_covid[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_covid["female"] = eat_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
eat_post["male"] = eat_post[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_post["female"] = eat_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

<ipython-input-230-210a951c5188>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-230-210a951c5188>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-230-210a951c5188>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-230-210

In [231]:
print(round(eat_pre["male"].mean()/5,2),"Pieces eaten on average per day for all male sharks before covid when feeding took place")
print(round(eat_pre["female"].mean()/7,2),"Pieces eaten on average per day for all female sharks before covid when feeding took place")
print(round(eat_covid["male"].mean()/5,2),"Pieces eaten on average per day for all male sharks when closed to the public when feeding took place")
print(round(eat_covid["female"].mean()/7,2),"Pieces eaten on average per day for all female sharks when closed to the public when feeding took place")
print(round(eat_post["male"].mean()/5,2),"Pieces eaten on average per day for all male sharks after the aquarium reopened when feeding took place")
print(round(eat_post["female"].mean()/7,2),"Pieces eaten on average per day for all female sharks after the aquarium reopened when feeding took place")

1.9 Pieces eaten on average per day for all male sharks before covid when feeding took place
1.54 Pieces eaten on average per day for all female sharks before covid when feeding took place
1.74 Pieces eaten on average per day for all male sharks when closed to the public when feeding took place
1.55 Pieces eaten on average per day for all female sharks when closed to the public when feeding took place
2.27 Pieces eaten on average per day for all male sharks after the aquarium reopened when feeding took place
1.72 Pieces eaten on average per day for all female sharks after the aquarium reopened when feeding took place


In [232]:
# Adding new calculations to original dataframes
eat_pre["All_GR"] = eat_pre['GR1'] + eat_pre['GR2'] + eat_pre['GR3'] + eat_pre['GR4'] + eat_pre['GR5']
eat_pre["All_BT"] = eat_pre['BT1'] + eat_pre['BT2'] + eat_pre['BT3'] + eat_pre['BT4'] + eat_pre['BT5']
eat_pre["All_SS"] = eat_pre['Ross'] + eat_pre['Chandler']
eat_covid["All_GR"] = eat_covid['GR1'] + eat_covid['GR2'] + eat_covid['GR3'] + eat_covid['GR4'] + eat_covid['GR5']
eat_covid["All_BT"] = eat_covid['BT1'] + eat_covid['BT2'] + eat_covid['BT3'] + eat_covid['BT4'] + eat_covid['BT5']
eat_covid["All_SS"] = eat_covid['Ross'] + eat_covid['Chandler']
eat_post["All_GR"] = eat_post['GR1'] + eat_post['GR2'] + eat_post['GR3'] + eat_post['GR4'] + eat_post['GR5']
eat_post["All_BT"] = eat_post['BT1'] + eat_post['BT2'] + eat_post['BT3'] + eat_post['BT4'] + eat_post['BT5']
eat_post["All_SS"] = eat_post['Ross'] + eat_post['Chandler']
eat_pre.head()

<ipython-input-232-77822da1387a>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-232-77822da1387a>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-232-77822da1387a>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-232-778

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,12.0,21.0,6.0,22.0,5.0
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,2.0,1.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,2.0,4.0,6.0,NaN,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,7.0,7.0,NaN,8.0,6.0
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,9.0,18.0,7.0,16.0,4.0


In [233]:
total_sharks_feeding_pre = eat_pre[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_pre = eat_pre[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_pre = eat_pre[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_pre = eat_pre[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_pre = eat_pre[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_pre = eat_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_feeding_pre

1      12
2      12
3       5
4       7
5      12
       ..
466    12
467     5
468     7
469    12
470    12
Length: 470, dtype: int64

In [234]:
total_sharks_feeding_covid = eat_covid[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_covid = eat_covid[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_covid = eat_covid[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_covid = eat_covid[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_covid = eat_covid[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_covid = eat_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_feeding_covid

472     7
473    12
474    12
475     5
476     7
477    12
478     5
479     7
480    12
481    12
482     5
483     7
484    12
485    12
486     5
487     7
488    12
489    12
490     5
491     7
492    12
493    12
494    12
495    12
496    12
497     7
498    12
499    12
dtype: int64

In [235]:
total_sharks_feeding_post = eat_post[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_post = eat_post[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_post = eat_post[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_post = eat_post[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_post = eat_post[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_post = eat_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_feeding_post

501     7
502    12
503    12
504     5
505     7
       ..
578    12
579    12
580     5
581     7
582    12
Length: 82, dtype: int64

In [239]:
eat_pre["All_GR"] = eat_pre["All_GR"] / GR_sharks_feeding_pre
eat_pre["All_BT"] = eat_pre["All_BT"] / BT_sharks_feeding_pre
eat_pre["All_SS"] = eat_pre["All_SS"] / SS_sharks_feeding_pre
eat_pre["female"] = eat_pre["female"] / female_sharks_feeding_pre
eat_pre["male"] = eat_pre["male"] / male_sharks_feeding_pre
eat_pre["Total Adjusted"] = eat_pre["Total: "] / total_sharks_feeding_pre
eaten_means_pre = pd.DataFrame(round(eat_pre.mean(), 3)).reset_index()
eaten_means_pre.columns = ["Subset","Mean"]
eaten_means_pre = eaten_means_pre.sort_values(by = "Mean", ascending = False)
prop_average = eaten_means_pre[eaten_means_pre["Subset"]=="Total Adjusted"]["Mean"].values[0]
eaten_means_pre = pd.merge(eaten_means_pre,pd.melt(eat_pre.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
#drop_means["color"] = "blue"
Average = []
for i in eaten_means_pre.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eaten_means_pre["Average"] = Average
max_y = (eaten_means_pre[1:]["Mean"] + eaten_means_pre["Standard Deviation"]).max()   
px.bar(eaten_means_pre.iloc[1:,:], x = "Subset", y = "Mean", color = "Average", error_y = "Standard Deviation",range_y = [0,max_y],title = "Eaten Means for each subset with 1 standard deviation before covid")

In [237]:
eat_covid["All_GR"] = eat_covid["All_GR"] / GR_sharks_feeding_covid
eat_covid["All_BT"] = eat_covid["All_BT"] / BT_sharks_feeding_covid
eat_covid["All_SS"] = eat_covid["All_SS"] / SS_sharks_feeding_covid
eat_covid["female"] = eat_covid["female"] / female_sharks_feeding_covid
eat_covid["male"] = eat_covid["male"] / male_sharks_feeding_covid
eat_covid["Total Adjusted"] = eat_covid["Total: "] / total_sharks_feeding_covid
eaten_means_covid = pd.DataFrame(round(eat_covid.mean(), 3)).reset_index()
eaten_means_covid.columns = ["Subset","Mean"]
eaten_means_covid = eaten_means_covid.sort_values(by = "Mean", ascending = False)
prop_average = eaten_means_covid[eaten_means_covid["Subset"]=="Total Adjusted"]["Mean"].values[0]
eaten_means_covid = pd.merge(eaten_means_covid,pd.melt(eat_covid.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
#drop_means["color"] = "blue"
Average = []
for i in eaten_means_covid.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eaten_means_covid["Average"] = Average
max_y = (eaten_means_covid[1:]["Mean"] + eaten_means_covid["Standard Deviation"]).max()   
px.bar(eaten_means_covid.iloc[1:,:], x = "Subset", y = "Mean", color = "Average", error_y = "Standard Deviation",range_y = [0,max_y],title = "Eaten Means for each subset with 1 standard deviation")

<ipython-input-237-c41aa4bc3b69>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-237-c41aa4bc3b69>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-237-c41aa4bc3b69>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-237-c41

In [238]:
eat_post["All_GR"] = eat_post["All_GR"] / GR_sharks_feeding_post
eat_post["All_BT"] = eat_post["All_BT"] / BT_sharks_feeding_post
eat_post["All_SS"] = eat_post["All_SS"] / SS_sharks_feeding_post
eat_post["female"] = eat_post["female"] / female_sharks_feeding_post
eat_post["male"] = eat_post["male"] / male_sharks_feeding_post
eat_post["Total Adjusted"] = eat_post["Total: "] / total_sharks_feeding_post
eaten_means_post = pd.DataFrame(round(eat_post.mean(), 3)).reset_index()
eaten_means_post.columns = ["Subset","Mean"]
eaten_means_post = eaten_means_post.sort_values(by = "Mean", ascending = False)
prop_average = eaten_means_post[eaten_means_post["Subset"]=="Total Adjusted"]["Mean"].values[0]
eaten_means_post = pd.merge(eaten_means_post,pd.melt(eat_post.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
#drop_means["color"] = "blue"
Average = []
for i in eaten_means_post.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eaten_means_post["Average"] = Average
max_y = (eaten_means_post[1:]["Mean"] + eaten_means_post["Standard Deviation"]).max()   
px.bar(eaten_means_post.iloc[1:,:], x = "Subset", y = "Mean", color = "Average", error_y = "Standard Deviation",range_y = [0,max_y],title = "Eaten Means for each subset with 1 standard deviation")

<ipython-input-238-1a402b2d37c3>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-238-1a402b2d37c3>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-238-1a402b2d37c3>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-238-1a4